## 导包

In [16]:
import os
from pdfminer.high_level import extract_text
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoConfig,  AutoTokenizer, AutoModel
from openai import OpenAI


## 获取所有文献的路径

In [3]:
root_dir = "./papers"
def extract_file_dirs(directory):
    file_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".pdf"):
                fp = os.path.join(root, file)
                file_paths.append(fp)
    return file_paths

files = extract_file_dirs(root_dir)
files

['./papers\\Enantioselective Iridium-Catalyzed Allylic Substitution with 2-Methylpyridines.pdf',
 './papers\\Iridium-Catalyzed Asymmetric Allylic Amination Reaction with N-Aryl Phosphoramidite Ligands.pdf']

## 获取其中某一篇文献的文字

In [4]:
texts = []
texts.append(extract_text(files[0]))
texts

['Communications\n\nAngewandte\n\nChemie\n\nAsymmetric Catalysis\n\nInternational Edition: DOI: 10.1002/anie.201700433\nGerman Edition:\nDOI: 10.1002/ange.201700433\n\nEnantioselective Iridium-Catalyzed Allylic Substitution with\n2-Methylpyridines\nXi-Jia Liu and Shu-Li You*\n\nAbstract: An enantioselective iridium-catalyzed allylic sub-\nstitution with a set of highly unstabilized nucleophiles gen-\nerated in situ from 2-methylpyridines is described. Enantioen-\nriched 2-substituted pyridines, which are frequently encoun-\ntered in natural products and pharmaceuticals, could be easily\nconstructed by this simple method in good yields and excellent\nenantioselectivity. The synthetic utility of the pyridine products\nis demonstrated through the synthesis of a key intermediate of\na reported Na+/H+ exchanger inhibitor and the total synthesis\nof ((cid:2))-lycopladine A.\n\nPyridines are among the most prevalent heterocyclic\n\nstructural moieties in biologically active natural products,\

In [ ]:
# 将从pdf中提取的文字写入文件
def write_to_file(text, file_name):
    with open(file_name, 'w', encoding='utf-8') as f:
        f.write(text)

# 循环调用函数
for i in range(0, len(texts)):
    file_name = f'text{i}.txt'
    write_to_file(texts[i], file_name)


In [ ]:
# 分析处理其中的 有问题的地方
# 哪些pdf转换有问题？改进转换方法，或者去除掉有问题的数据


## 切分文献

In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
splits = []
for text in texts:
    splits.extend(text_splitter.split_text(text))
splits[:5], len(splits)

(['Communications\n\nAngewandte\n\nChemie\n\nAsymmetric Catalysis\n\nInternational Edition: DOI: 10.1002/anie.201700433\nGerman Edition:\nDOI: 10.1002/ange.201700433\n\nEnantioselective Iridium-Catalyzed Allylic Substitution with\n2-Methylpyridines\nXi-Jia Liu and Shu-Li You*\n\nAbstract: An enantioselective iridium-catalyzed allylic sub-\nstitution with a set of highly unstabilized nucleophiles gen-\nerated in situ from 2-methylpyridines is described. Enantioen-\nriched 2-substituted pyridines, which are frequently encoun-\ntered in natural products and pharmaceuticals, could be easily\nconstructed by this simple method in good yields and excellent\nenantioselectivity. The synthetic utility of the pyridine products\nis demonstrated through the synthesis of a key intermediate of\na reported Na+/H+ exchanger inhibitor and the total synthesis\nof ((cid:2))-lycopladine A.\n\nPyridines are among the most prevalent heterocyclic',
  'Pyridines are among the most prevalent heterocyclic\n\nstr

## 将切分的文献保存进向量数据库

In [22]:
# embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-small-zh-v1.5")

# 保存到当前根目录的vector_db文件夹下
kb_name = "vector_test2"
persist_directory = os.path.join(os.getcwd(), kb_name)

vectordb = Chroma.from_texts(
    texts=splits,
    embedding=embedding,
    persist_directory=persist_directory,
)
vectordb.persist()

## 构造关于这篇文献的相关问题

In [18]:
query4 = '请回答关于此篇文献中有关于化学反应的优化相关的问题，说明清楚如催化剂/配体/添加剂/溶剂/温度/时间/空气等的实验条件，对反应产率或者选择性产生了哪些影响，并给出关于此化学反应的清晰明确的优化建议。'
query7 = '请说明此处讨论的化学反应机理的推导过程，并且列举出化学反应的实验结果和实验结论。'

In [23]:
question = "文章中主要是围绕什么样的化学反应在进行讨论？" + query7
q_docs = vectordb.similarity_search(question, k=1)
context = [doc.page_content for doc in q_docs]

prompt = f"已知PDF内容：\n{'-'.join(context)}\n根据已知信息回答问题：\n{question}\n所有的回答都根据已知信息的内容来回答，不要编造内容。用中文回答问题。"
prompt

'已知PDF内容：\nenantioselectivity. The synthetic utility of the pyridine products\nis demonstrated through the synthesis of a key intermediate of\na reported Na+/H+ exchanger inhibitor and the total synthesis\nof ((cid:2))-lycopladine A.\n根据已知信息回答问题：\n文章中主要是围绕什么样的化学反应在进行讨论？请说明此处讨论的化学反应机理的推导过程，并且列举出化学反应的实验结果和实验结论。\n所有的回答都根据已知信息的内容来回答，不要编造内容。用中文回答问题。'

## 通过调用模型api接口获得模型输出

In [ ]:
os.environ["OPENAI_API_KEY"] = "ab78b1a21a614ba1ade36ba88b172893"


In [24]:
client = OpenAI(
    api_key='ab78b1a21a614ba1ade36ba88b172893',
    base_url='https://api.lingyiwanwu.com/v1'
)
completion = client.chat.completions.create(
    model="yi-large",
    messages=[{"role": "user", "content": prompt}]
)

answer = completion.choices[0].message.content
print(completion.choices[0].message.content)

根据您提供的PDF内容，文章主要围绕着吡啶类化合物的合成以及其在合成其他化合物中的应用进行讨论。吡啶是一种含氮的六元杂环化合物，在有机合成中非常重要，因为它可以作为多种药物和生物活性分子的合成前体。

化学反应机理的推导过程通常涉及以下几个步骤：
1. 反应物和试剂的识别：首先需要确定参与反应的分子和可能的反应条件。
2. 反应中间体的形成：根据已知的化学反应原理，推测可能形成的反应中间体，如自由基、离子或配合物等。
3. 关键步骤的确定：分析反应中决定产率和选择性的关键步骤，这通常涉及到立体选择性或区域选择性的形成。
4. 产物的生成：从反应中间体到最终产物的转化过程。

由于您提供的内容有限，无法详细描述具体的反应机理。通常，吡啶的合成可以通过多种方法实现，包括多组分反应、环化反应等。在合成过程中，立体选择性是一个重要的考虑因素，因为它直接影响到产物的手性纯度。

实验结果和实验结论通常包括：
- 产物的产率和纯度：通过实验确定合成产物的收率和纯度。
- 立体选择性：如果反应涉及到手性中心，需要评估产物的光学纯度或对映体过量（ee值）。
- 应用实例：展示合成出的吡啶化合物如何用于合成其他生物活性分子或药物。

在您提供的PDF内容中，提到了合成出的吡啶化合物被用于合成一种Na+/H+交换抑制剂的关键中间体，以及完成了((cid:2))-lycopladine A的全合成。这些应用实例展示了吡啶化合物的合成在药物化学和天然产物合成中的重要性。

由于缺乏具体的实验数据和详细的反应条件，无法提供更具体的实验结果和结论。如果您能提供更多的信息，我可以尝试给出更详细的答案。


## 先把文字保存为txt文件

In [ ]:
# 将 answer 保存run_answer.txt
with open('run_answer.txt', 'w') as f:
    f.write(answer)

## 再将所有的txt文件一起保存进 向量数据库中

In [ ]:
# 将 run_answer.txt 中的内容读取出来 并保存到变量中
with open('run_answer.txt', 'r') as f:
    content = f.read()

## 将模型回答保存进向量数据库中作为高质量语料

In [25]:
# 创建一个新的向量数据库，存储得到的回答
# 保存到当前根目录的vector_answers文件夹下
kb_name = "vector_answers"
persist_directory = os.path.join(os.getcwd(), kb_name)

answers = [answer]

vector_question4 = Chroma.from_texts(
    texts=answers,
    embedding=embedding,
    persist_directory=persist_directory,
)
vectordb.persist()

至此，通过对某一篇文献提问，获取得到的答案作为语料存入向量库中，以便回答相关问题。

接下来，需要做的
1. 调整文档片段的召回方式，使得召回的文档片段数量尽可能多，同时保证召回的文档片段的相似度尽可能高。以便LLM能够更充分地理解文档片段，并且回答相关问题。
2. 调整代码，能够通过循环的方式回答一篇文章的所有问题。
3. 调整代码，能够通过循环的方式回答完所有的文章，并且将结果保存到文件中。
4. 将保存的txt文件中的答案读取出来，存入向量数据库中。